In [1]:
%matplotlib inline
import mpld3
mpld3.enable_notebook()
from matplotlib import pyplot as plt
plt.rcParams['figure.figsize'] = [12, 8]
plt.rcParams['figure.dpi'] = 100

import numpy as np
from random import random, uniform, randrange, triangular
from tqdm import tqdm

In [2]:
#Pytorch imports
import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader
from torch.autograd import Variable
from onnx import optimizer

In [3]:
# base_size = prev_data//group_base
# IMU_size = prev_data//group_IMU
# torque_size = (prev_data + fwd_data)//group_base
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(200, 64),
            nn.ReLU(),
            nn.Linear(64, 64),
            nn.ReLU(),
            nn.Linear(64, 10),
            nn.Tanh()
        )

        
    def forward(self, x):
        return self.linear_relu_stack(x)

dummy = torch.zeros((1, 200,))
input_var = Variable(dummy)
model = torch.load('model_L1Norm.pth')
print(model)
m_inp = torch.ones((1, 200))
print(model(m_inp))

NeuralNetwork(
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=200, out_features=64, bias=True)
    (1): ReLU()
    (2): Linear(in_features=64, out_features=64, bias=True)
    (3): ReLU()
    (4): Linear(in_features=64, out_features=10, bias=True)
    (5): Tanh()
  )
)
tensor([[0.7225, 0.7538, 0.8294, 0.8637, 0.8681, 0.8234, 0.8683, 0.8890, 0.9146,
         0.8543]], grad_fn=<TanhBackward>)


In [4]:
from pytorch2keras import pytorch_to_keras
# we should specify shape of the input tensor
k_model = pytorch_to_keras(model, input_var, verbose=False)  
k_model.compile()

In [5]:
from tensorflow import keras
k_model.save('model_s_keras.h5')
# print(k_model.shape)
reconstructed_model = keras.models.load_model("model_s_keras.h5")
print(reconstructed_model.summary())

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_0 (InputLayer)         [(None, 200)]             0         
_________________________________________________________________
7 (Dense)                    (None, 64)                12864     
_________________________________________________________________
8 (Activation)               (None, 64)                0         
_________________________________________________________________
9 (Dense)                    (None, 64)                4160      
_________________________________________________________________
10 (Activation)              (None, 64)                0         
_________________________________________________________________
11 (Dense)                   (None, 10)                650       
_________________________________________________________________
output_0 (Activation)        (None, 10)                0     

In [8]:
import konverter
konverter.konvert('model_s_keras.h5', output_file='export/model_L1')


Successfully got model architecture! 😄

Layers:
  name: InputLayer (ignored)
  name: dense
  activation: relu
  shape: (200, 64)
  name: Activation (ignored)
  name: dense
  activation: relu
  shape: (64, 64)
  name: Activation (ignored)
  name: dense
  activation: tanh
  shape: (64, 10)
  name: Activation (ignored)

🔨 Now building pure Python + NumPy model...
🙌 Saved Konverted model!
Output model file: export/model_L1.py
Weights and biases file: export/model_L1_weights.npz

❗ Make sure to change the path inside the wrapper file to your weights if you move the file elsewhere.
